In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc

In [11]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Prepping Data for algorithm

In [3]:
X = pd.read_csv('../input/final_data.csv')
Y = pd.read_csv('../input/final_target.csv', header=None)

In [4]:
Y = Y[1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=18)

In [6]:
train_lgb = lgb.Dataset(data=X_train, label=y_train)

# Defining Search Space

In [22]:
space  = [Integer(3, 10, name='max_depth'),
          Integer(5, 50, name='num_leaves'),
          Real(0, 0.5, name='reg_alpha'),
          Real(0.6, 0.9, name='subsample'),
          Real(0.6, 0.9, name='colsample_bytree'),
          Real(0, 0.5, name='reg_lambda'),
          Real(0.01, 0.02, name='learning_rate'),
          Real(0.01, 0.03, name='min_split_gain'),
          Integer(1, 40, name='min_child_weight')
         ]

# Defining Objective function to optimize

In [23]:
def objective_func(values):
    params = {
        'task': 'train', 
        'boosting_type': 'gbdt', 
        'objective': 'binary', 
        'metric': 'auc', 
        'learning_rate': values[6], 
        'num_leaves': values[1], 
        'verbose': 0 ,
        'colsample_bytree':values[4],
        'subsample':values[3], 
        'max_depth':values[0], 
        'reg_alpha':values[2], 
        'reg_lambda':values[5], 
        'min_split_gain':values[7], 
        'min_child_weight':values[8]
    }
    print('----------------------------------------------------------------------------------------')
    print('Current Parameters : ', params)
    model = lgb.train(params, train_lgb,500)
    predictions_lgb = model.predict(X_test)
    auc = - roc_auc_score(y_test, predictions_lgb)
    print('Current Score : ', -auc)
    gc.collect()
    return auc

# Running Optimization Algorithm

In [ ]:
result_gp = gp_minimize(objective_func, space, n_calls=50, random_state=0, n_random_starts=10)

----------------------------------------------------------------------------------------
Current Parameters :  {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.012975346065444723, 'num_leaves': 43, 'verbose': 0, 'colsample_bytree': 0.7870691090357917, 'subsample': 0.8541755216352377, 'max_depth': 7, 'reg_alpha': 0.4289728088113785, 'reg_lambda': 0.19219085364634997, 'min_split_gain': 0.011134259546348864, 'min_child_weight': 12}
